# mpH<sup>2</sup>MM Tutorial

<p class="lead">This <a href="https://jupyter.org/">Jupyter notebook</a>
To run a cell, press <b>'shift+enter'</b> while the cell is selected. Before running, make sure you have installed <a href="https://numpy.org/">NUMPY</a>, <a href="https://matplotlib.org">Matplotlib</a>, and <a href="https://github.com/harripd/H2MMpythonlib/">H2MM_C</a></p>

NUMPY is one of the most commonly used python modules in scientific computing, so it might be included already in your install, but if it is not, you can follow the installation guide here: <a href="https://numpy.org/install/">https://numpy.org/install/</a>

Matplotlib is a very common python module for creating professional looking plots and graphs, and like NUMPY, is very common in scientific applications, like NUMPY it may even be included by default in your python installation. If it is not, see the install guide here: <a href="https://matplotlib.org/stable/users/installing.html">https://matplotlib.org/stable/users/installing.html</a>


If you are new to Python, and want some tutorials, may I recomend the tutorials on <a href="https://www.youtube.com/user/schafer5">Corey Schafer's YouTube channel</a>. Some of the particular videos I'd recomend are the following: **lists tuple and sets**, **Loops and iterations**, **functions**.

Cells can be 'Code' 'Markdown' 'Raw NBConvert' or 'Heading'
- Code cells are the "meat" of the jupyter notebook, they are where you put python code you actually want to run
- Markdown cells are for communicating with the reader, they use the github flavor of Markdown, so when they are rendered, (either when you open the notebook, or after editing you can hit shift+enter) the text is readable, and has various formatting niceties like bold, italics, headings, bullet points etc.
- Raw NBConvert cells are rare, not used in this tutorial, but contain data that is neither rendered or executed except when exporting to PDF or the like
- Heading, a soon to be deprecated feature, which gives you a heading in the Markdown format (ie a heading in a markdown cell is equivalent to a heading cell)



# Outline
0. [Importing Modules](#0)
1. [Basic Analysis](#1)
 - [1.1 Basic Analysis](#11)
     - [1.1.1 Load Data](#111)
     - [1.1.2 Define some initial "guess" models](#112)
     - [1.1.3 Optimize Model](#113)
 - [1.2 Viterbi](#12)
 - [1.3 Likelihood Estimators](#13)
 - [1.4 Analysis Workflow](#14)
 - [1.5 `H2MM_arr` Function](#15)
2. [The `h2mm_model` object](#2)
 - [2.1 Accessing core `h2mm_model` parameters](#21)
 - [2.2 Review of `h2mm_model` Fields](#22)
 - [2.3 Changing Values in a Model](#23)
 - [2.4 `h2mm_model` Methods](#24)
     - [2.4.1 The `.copy()` method](#241)
     - [2.4.2 The `.evaluate` Method](#242)
     - [2.4.3 The `.optimize` Method](#243)
     - [2.4.4 `EM_H2MM_C` vs `.optimize` and `H2MM_arr` vs `.evaluate`: They're basically the same](#244)
3. [Format of Color and Time (Data)](#3)
4. [Multiparameter H2MM](#4)
 - [4.1 Working with more than 2 streams](#41)
 - [4.2 Setting up the **emission probability matrix**](#42)
 - [4.3 Checking a Model: `.ndet` and `.nstate` fields in `h2mm_model` objects](#43)
5. [Controlling `EM_H2MM_C`](#5)
 - [5.1 Convergence](#51)
     - [5.1.1 Using `.optimize` instead](#511)
 - [5.2 Limits](#52)
     - [5.2.1 Our First Bounded Fit](#521)
     - [5.2.2 The `h2mm_limits` object](#522)
     - [5.2.3 `bounds_func` Options](#523)
 - [5.3 Auto-generate Initial Model with `factory_h2mm_model` function](#53)
     - [5.3.1 Use `factory_h2mm_model` to make initial model](#531)
     - [5.3.2 Bounds and `factory_h2mm_model`](#532)
     - [5.3.3 Distribution kwargs in `factory_h2mm_model`](#533)
     - [5.3.4 Full analsis with bounds and `factory_h2mm_model`](#534)
 - [5.4 "Continuous" Analysis](#54)
 - [5.5 While loop analysis](#55)
6. [Customizing `EM_H2MM_C`](#6)
 - [6.1 Changing parallelization: the `num_cores` kwarg](#61)
 - [6.2 Printing `EM_H2MM_C` iterations](#62)
     - [6.2.1 Basic `print_func` options](#621)
     - [6.2.2 Customized display using your own function](#622)
 - [6.3 User Defined Limits Function](#63)
7. [Apendices](#7)
 - [`h2mm_model` fields](#71)
     - [7.1.1 Statistical Fields](#711)
     - [7.1.2 `.niter`](#712)
     - [7.1.3 Convergence Info](#713)
 -[7.2 `EM_H2MM_C` and `.optimize` Arguments and Fields](#72)
     - [7.2.1 The `reset_niter` argument](#721)

Cells that are examples of "complete" analysis are preceeded by headings in <font color="blue">blue</font>. The fast-learning/experience Python programer might be able to piece everything togoether with just these cells. These cells will also be useful as "skeletons" for building analysis of other data. (Please use as a skeleton, not just copy paste)

- [Optimizing a model](#optimization)
- [Basic Workflow](#wholeanalysis1)
- [Error Evaluation](#erroreval) Note: this does not include the prior optimization steps
- [Analysis using auto-generated models](#fhmwrkflow) Note: this and the next blue cell are very similar
- [Analysis with bounds and autogenerated models](#534)
- [Continuous Analysis Worklfow](#optwrkflow)
- [While loop based Workflow](#finaldemo)


<a id='0'></a>
# 0. Importing Modules

First, the Import the modules, we're going to need:
- `numpy` for some array inputs, we use `import numpy as np` so that whenever we use numpy we can call it with the prefix `np.`. This is standard practice in scientific computing.
- `matplotlib` for plotting graphs so we can visuallize our results, we use `from matplotlib import pyplot as plt` because it turns out that we'll only need the `pyplot` subset of `matplotlib`, and `plt` is much less wordy than typing `matplotlib.pyplot` over and over. Like how we imported `numpy`, this is standard practice in scientific computing. 
- `H2MM_C` the API for mpH2MM, we use `from H2MM_C import *` so all functions/classes are available specified without an addition `H2MM_C.` identifier

We're also importing
- `os` so we can load our sample datafile, its name is alrady so short, we just call it as `import os`
    - in real analysis, this will likely be replaced by a module like <a href="https://fretbursts.readthedocs.io/en/latest/">FRETbursts</a>, which will analyze the raw data.
    - Often in real analysis you will still need many of the functions in `os`, so you'll probably import `os` anyways, even if you are using FRETBursts

In [1]:
#import the needed modules
import os
import numpy as np
from matplotlib import pyplot as plt

from H2MM_C import *

<a id='1'></a>
# 1. Basic Analysis

This tutorial is going to follow a basic format: First we'll introduce an idea/function, give a basic example, and then dive into the details of what everything is.

mpH<sup>2</sup>MM is fundamentally an optimization method. It's main goal is to find the "optimal" **model** that describes a set of **data**. The **model** assumes the system is transitioning between a finite number of **states**. It is further assumed that the **states** are "hidden" and that the data instead comes in the form of a set of data points, which we will call **photons** because that is what they are when we are working with single molecule confocal data, the system for which H<sup>2</sup>MM was designed, which are each have a **time** and **detector** (or **stream**) and each state has a certain probability of producing a photon, in each **stream**. Usually you will have many short sequences that all represent the same system, which we will refer to as **bursts**, again, because these usually represent bursts of photons caused by single molecules of the same sample diffusing in and out of the confocal volume.

Each **model** is composed of a set of arrays:
1. The **initial probability matrix**, this is the least important array, which describes the proability of the data beginning in each state.
2. The **transition probability matrix**, the likelihood of one state transitioning to another
3. The **emission probability matrix**, the likelihood of a state producing a data point in a given stream

"Optimization" is specifically the optimization of the logliklihood, basically, it is the likelihood that the model describes the data, so optimization is a process to find the model that has the highest, i.e. the best chance of describing the data.

<a id='11'></a>
## 1.1 Run Analysis
So, the basic process is:
1. Load and/or process data
2. Define an initial **model**
3. Optimize **model** against **data**

Since mpH<sup>2</sup>MM is usually used for confocal burst data, the **streams** are the detectors, and data is usually composed of a set of bursts of photons. So the first step is usually taking your burst selection, and converting it into a form that `H2MM_C` can accept.

<a id='111'></a>
### 1.1.1 Load Data
#### First, we need some data, let's load it from the provided file:
- this is just a sample data for us to work with, you'd usually use a module like <a href="https://fretbursts.readthedocs.io/en/latest/">FRETBursts</a> to process the raw data and slice it into bursts
- if you are using Matlab for your initial data analysis, then you will want to export the data to an ascii text file, and load it with the bellow function (make sure you get the format correct, especially note that python indexes from 0, while Matlab indexes from 1, so you want to make sure that photon streams are indexed as 0,1,2... and not as 1,2,3...
The **data** is imported into two lists: `color` and `times`
- `color` is a list* (it can also be a tuple) of NUMPY arrays, each array is a **burst** which contains the detector indices (0,1,2...) of the photons
- `times` is a list* (it can also be a tuple) of NUMPY arrays, much like in `color` 1 array 1 burst, but instead of containing the detectors, these are the photon arrival times (arbitrary units)
So make sure that the elements of `color` and `times` match, i.e. they're the same length, we'll be more specific later.

 *For those coming from Matlab, Python lists are very similar to Matlab Cell Arrays, Python tuples are similar, except that they cannot be changed after their creation

### Load Bursts:

In [2]:
# This cell will only be used if you export your data to an ascii file, 
# if you are using a software package such as FRETBursts, a different
# function (for FRETBursts, look at our other Jupyter Notebooks, it is
# the data_sort function)
times = []
color = []
i = 0
# load the data
with open('sample_data_2det.txt') as f:
    for line in f: # this reads each line one at a time
        if i % 2 == 0: # even lines are times
            times.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else: # odd lines are color
            color.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1
print("Done loading data")

Done loading data


<a id='112'></a>
### 1.1.2 Define some initial "guess" models

In `H2MM_C` the **model** is a special object: the `h2mm_model` object (for those coming from Matlab, think of it like a structure with predefined fields, but remember it has a lot more functionality built into it than a Matlab structure).
There are 3 components to the model, all are specified as NUMPY arrays, with a floating-point data type. They are all row stochastic, ie each row sums to 1, and there are no negative values:
- **Initial Probability**: (`.prior`) the probability of the system being in each state
    - A 1D array, with length of the number of states
- **Transition Probability matrix**: (`.trans`) each element `[i,j]` is the probability of the system transitioning from state `i` to state `j` in one time step (ie the unit of time used as 1 in the times array)
    - A 2D square array, with dimensions of the number of states
    - Usually the values on the diagonal are close to 1, and off-diagonal close to 0
- **Emission Probability matrix**: (`.obs`) the probability that a photon will arise from a given photon stream, ie the `[i,j]` element is the probability that, given the system is in state `i`, a photon will belong to photon stream `j`
    - A 2D array, with dimension 0 being the number of states, and dimension 1 being the number of photon streams

A final note: when imported into a model, all these arrays are *row stochastic* that is each row sums to 1. It is highly recommended to use as inputs *row stochastic* matrices.

#### Define Model Arrays:

In [3]:
# first we're going to define the arrays
prior = np.array([0.5,0.5]) # note: it is a 1D array, and has 2 elements, that means this model has 2 states
trans = np.array([[0.999999, 1e-6],[1e-6, 0.999999]]) # note that it is 2D, and square, with 2 states
obs = np.array([[0.3, 0.7],[0.5, 0.5]]) # note that it is 2D, with 2 states (rows) and 2 streams (columns)

Now we declare the initial model, the class of a model is `h2mm_model`, and the call to initiallize an object of this class is of the format `h2mm_model(prior,trans,obs)`
#### Make Model Object

In [4]:
# make the 2 state model
init_model = h2mm_model(prior,trans,obs)

<a id='113'></a>
### 1.1.3 Optimize Model

The core function of the `H2MM_C` API is called `EM_H2MM_C`, which takes 3 inputs:
- The initiating `h2mm_model`, (`init_model` in this example)
- The list or tuple of NUMPY arrays of photon streams, `color` in this example
- the list or tuple of NUMPY arrays of arrival times, `times` in this example

The function outputs a new, optimized `h2mm_model`

The basic signature is `EM_H2MM_C(model,color,time)`

### Run the optimization

In [5]:
# run the optimization
opt_model = EM_H2MM_C(init_model,color,times)

The model converged after 58 iterations


### Examining the result

You can print everything out in a jupyter notebook running a cell that has only the naem of the output variable: 

*For those who like to get deep into Python, this calls the `.__repr__()` method*

In [6]:
opt_model

nstate: 2, ndet: 2, nphot: 239701, niter: 58, loglik: -120420.61071616189 converged state: 3
prior:
0.26413235986013317, 0.7358676401398668
trans:
0.9999780436287965, 2.1956371203566618e-05
7.02884750921139e-06, 0.9999929711524908
obs:
0.3344060168725045, 0.6655939831274955
0.855176043581418, 0.14482395641858206

<a id='optimization'></a>
### <font color='blue'>Here we see the whole analysis in one cell:</font>

In [7]:
# load data
times = []
color = []
i = 0
# open the file
with open('sample_data_2det.txt') as f:
    for line in f: # this reads each line one at a time
        if i % 2 == 0: # the datafiel is formate
            times.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1

# first we're going to define the arrays
prior = np.array([0.5,0.5])
trans = np.array([[0.999999, 1e-6],[1e-6,0.999999]])
obs = np.array([[0.3, 0.7],[0.5, 0.5]])
init_model = h2mm_model(prior,trans,obs)

# optimize the model
opt_model = EM_H2MM_C(init_model,color,times)
# print optimized model
opt_model

The model converged after 58 iterations


nstate: 2, ndet: 2, nphot: 239701, niter: 58, loglik: -120420.61071616135 converged state: 3
prior:
0.2641323598601337, 0.7358676401398664
trans:
0.9999780436287965, 2.1956371203566655e-05
7.028847509211437e-06, 0.9999929711524908
obs:
0.3344060168725048, 0.6655939831274952
0.8551760435814182, 0.14482395641858192

<a id='12'></a>
## 1.2 Viterbi

Now that a model has been found, it can be useful to find the *most likely* state of each **photon** in a **burst**. This is achieved using the *Viterbi* algorithm.

There are 2 levels of *Viterbi* algorithm analysis: `viterbi_path` and `viterbi_sort`.
`viterbi_sort` is a wrapper around `viterbi_path` which returns more outputs, processing the data based on dwells.
Both take the same core inputs as `EM_H2MM_C`, that is `viterbi_path(model,color,times)` and `viterbi_path(model,color,times)`.
The basic outputs of `viterbi_path` are (in order)
- *Path*: the most likely state path through the data- a list of 1D int numpy arrays
- *Scale*: the posterior probability of each photon, ie, the likelihood of the given photon belonging to the state given in `path`- a list of 1D float numpy arrays
- *Loglik* the logliklihood of each path, given as a 1D NUMPY array, with each element cooresponding to a burst.
- *ICL*: the integrated complete likelihood of the path through all the data, basically BIC of the *Viterbi* path

The outputs of `viterbi_sort` are reordered slightly, with the ICL put first, and outputs placed after the outputs of `viterbi_path`see the docstring for more details (run `viterbi_sort?`).

#### Run *Viterbi* algorithm

In [8]:
# run viterbi_path on optimized model
path, scale, loglik, icl = viterbi_path(opt_model,color,times)

Now, let's look at an element of `path` to see what it looks like

In [9]:
path[0]

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint64)

Also note that each array in `path` and `scale` match the size of the equivalent arrays in `color` and `times`

In [10]:
for p, s, t in zip(path,scale,times):
    assert p.size == s.size == t.size
print("All arrays match size of equivalent arrays in times")

All arrays match size of equivalent arrays in times


<a id='13'></a>
## 1.3 Likelihood Estimators

It was mentioned that `EM_H2MM_C` maximizes the *loglikelihood* of a model given a set of data. This is a field in the `h2mm_model` object, simply named `.loglik`.

*NOTE: when an initiating model is made with `h2mm_model(prior,trans,obs)` method, the initial model has a `.loglik` of `-inf`.*

In [11]:
opt_model.loglik

-120420.61071616135

Remember that it was mentioned that the optimization of `EM_H2MM_C` doesn't change the number of states? This is important, because unless you know for sure that there are n number of states, models with different numbers of states should be optimized and then compared.

So use it is generally better to proceed as follows:
 1. Define several **models** with different numbers of **states**
 2. Run optimization on **data** using all **models**
 3. Compare optimized **models** and select the best one

How to compare the models? Don't just choose the model you "like," rather, it's best to use a statistical discriminator (or several). The *loglikelihood* however, is not a good discriminator. Generally speaking, it is good only for comparing models with the same number of states. Models with more states basically always have better *loglikelihoods.* But there are ways to make working statistical discriminators. 

There are two that `H2MM_C` has built-in: the $BIC$ and $ICL$

<a id='131'></a>
### 1.3.1 Bayes Information Criterion

The *Bayes Information Criterion* ($BIC$), which penalizes the *loglikelihood* based on the number of free parameters (related to the nuber of *states*, and factoring in the number of **streams**.

Here is the statistical definition of the $BIC$:

$ BIC = -2 LL + K \ln(n)$

Where $LL$ is the *loglikelihood*, $K$ is the number of free parameters, and $n$ is the number of **photons** in the data set.

Since the $BIC$ is based purely on the *loglikelihood* and the data against which the **model** was optimized, the $BIC$ is a field available in the `h2mm_model`: accessed through `.bic`

<a id='132'></a>
### 1.3.2 Integrated Complete Likelihood

The *Integrated Complete Likelihood* ($ICL$) is basically the same, but based on the *loglikelihood* not of the **model**, but rather of the *most likely state path* based on the *Viterbi* algorithm.

$ ICL = -2 LL_{Viterbi} + K \ln(n)$

We find the $ICL$ is a better discriminator that $BIC$. Unfortunately, unlike the $BIC$, since $ICL$ is based on the *most likely state path*, it requires running `viterbi_path` or `viterbi_sort`, and is therefore not a field in the `h2mm_model` object.

So, let's see how to access and calculate both of these (if you were paying close attention, you'll already know how to find the $ICL$)

#### Finding the BIC:

In [12]:
# the BIC:
opt_model.bic

240903.15717028416

### And the ICL
(remember, we have to run *Viterbi*)

In [13]:
# calculate viterbi, the ICL is the last output
path, scale, loglik, icl = viterbi_path(opt_model,color,times)
# print the ICL:
icl

247597.7196126414

<a id='14'></a>
## <font color="blue">1.4 Analysis Workflow</font>

Now we have all the tools we need to properly analyze a set of data, and decide what **model** best describes it:
1. Load and/or process data (we recomend using *FRETBursts*)
2. Define several initial **models** with different numbers of states
3. Optimize each **model** against the **data**
4. Calculate $ICL$ with the *Viterbi* algorithm
5. Compare $ICL$ (and potentially $BIC$) to select the best model, the minimum is considered best

<a id='wholeanalysis1'></a>
#### <font color="blue">So, let's see this in action:</font>

In [ ]:
# load data (this is the same as before)
times = []
color = []
i = 0
# open the file
with open('sample_data_2det.txt') as f:
    for line in f: # this reads each line one at a time
        if i % 2 == 0: # the datafiel is formate
            times.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1


# define several models, with different numbers of states (this is different from previous cells)
# 1state model
prior_1state = np.array([1.0])
trans_1state = np.array([[1.0]])
obs_1state = np.array([[0.5,0.5]])
init_model_1state = h2mm_model(prior_1state,trans_1state,obs_1state)
# 2 state model
prior_2state = np.array([0.5,0.5])
trans_2state = np.array([[0.999999, 1e-6],[1e-6, 0.999999]])
obs_2state = np.array([[0.3, 0.7],[0.5, 0.5]])
init_model_2state = h2mm_model(prior_2state,trans_2state,obs_2state)

# 3 state model
prior_3state = np.array([0.3, 0.3, 0.4])
trans_3state = np.array([[0.999998,1e-6,1e-6],[1e-6,0.999998,1e-6],[1e-6,1e-6,0.999998]])
obs_3state = np.array([[0.3,0.7],[0.5,0.5],[0.7,0.3]])
init_model_3state = h2mm_model(prior_3state,trans_3state,obs_3state)

# 4 state model
prior_4state = np.array([0.2,0.2,0.2,0.4])
trans_4state = np.array([[0.999997,1e-6,1e-6,1e-6],[1e-6,0.999997,1e-6,1e-6],[1e-6,1e-6,0.999997,1e-6],[1e-6,1e-6,1e-6,0.999997]])
obs_4state = np.array([[0.2,0.8],[0.4,0.6],[0.6,0.4],[0.8,0.2]])
init_model_4state = h2mm_model(prior_4state,trans_4state,obs_4state)


# Optimize each model
opt_model_1state = EM_H2MM_C(init_model_1state,color,times)
opt_model_2state = EM_H2MM_C(init_model_2state,color,times)
opt_model_3state = EM_H2MM_C(init_model_3state,color,times)
opt_model_4state = EM_H2MM_C(init_model_4state,color,times)


# calculate ICL
path_1state, scale_1state, ll_1state, icl_1state = viterbi_path(opt_model_1state,color,times)
path_2state, scale_2state, ll_2state, icl_2state = viterbi_path(opt_model_2state,color,times)
path_3state, scale_3state, ll_3state, icl_3state = viterbi_path(opt_model_3state,color,times)
path_4state, scale_4state, ll_4state, icl_4state = viterbi_path(opt_model_4state,color,times)

The model converged after 2 iterations
The model converged after 58 iterations


Now, let's examine the results, first, let's look at the $BIC$:

In [ ]:
print(f"1 state BIC = {opt_model_1state.bic}")
print(f"2 state BIC = {opt_model_2state.bic}")
print(f"3 state BIC = {opt_model_3state.bic}")
print(f"4 state BIC = {opt_model_4state.bic}")

And then the $ICL$:

In [ ]:
print(f"1 state BIC = {icl_1state}")
print(f"2 state BIC = {icl_2state}")
print(f"3 state BIC = {icl_3state}")
print(f"4 state BIC = {icl_4state}")

Let's use Matplolib to graphically represent this:

In [ ]:
plt.scatter([1,2,3,4],[opt_model_1state.bic, opt_model_2state.bic, opt_model_3state.bic, opt_model_4state.bic])
plt.title("BIC")

In [ ]:
plt.scatter([1,2,3,4],[icl_1state, icl_2state, icl_3state, icl_4state])
plt.title("ICL")

The $ICL$ of the 2 state model is at a minimum, therefore it is most likely the ideal model. 

The $BIC$ shows the oposite patern, but in our testing, the $BIC$ usually always improves with more states, however, the improvement decreases with succesive states- so you might choose a threshold, or like *Lerner2018*, use a modified $BIC$.

*We've also noticed that the principle of the more data the better is especially true for H<sup>2</sup>MM analysis. So if you're seeing plateaus, espeically in $ICL$, consider aquiring more data to make things clear.*

So, let's see the ideal model:

In [ ]:
opt_model_2state

<a id='15'></a>
## 1.5 `H2MM_arr` Function

Sometimes you don't want to optimize a model, but rather just get the *loglikelihood* or $BIC$ out of that model. Maybe you want to get a bunch of loglikelihoods against the same set of data.

For this, there is the `H2MM_arr` function, which has a very similar signature to that of `EM_H2MM_C`,  `viterbi_path`, and `viterbi_path`: `H2MM_arr(models,color,time)` The main difference is that `models` can be a `list`, `tuple`, NUMPY array of `h2mm_model` objects, or just a `h2mm_model` object.

#### This can be used as a way to see how sharply the *loglikelihood* peaks around it's optimal value, which can help you estimate error bars

To do this, we'll first extract the model parameters, (`.prior`, `.trans`, and `.obs`) into separate arrays, and make several initial models from those, each with one values slightly tweaked (and make sure to renormalize the particular array), and then evaluate the *loglikelihood* with `H2MM_arr`:

#### <font color="gray">Setup the new models</font>

In [ ]:
# extract prior, trans and obs
prior_opt = opt_model_2state.prior
trans_opt = opt_model_2state.trans
obs_opt = opt_model_2state.obs

# make copies of the array so the tweak can be applied
# we need to do "+ 0" so that the tweak arrays are separate copies in memory
trans_tweak1 = trans_opt + 0
trans_tweak2 = trans_opt + 0
trans_tweak3 = trans_opt + 0
trans_tweak4 = trans_opt + 0

# let's tweak the value of trans[0,1], lets adjust it by 1e-7 up and down
trans_tweak1[0,1] = trans_opt[0,1] - 2e-7
trans_tweak2[0,1] = trans_opt[0,1] - 1e-7
trans_tweak3[0,1] = trans_opt[0,1] + 1e-7
trans_tweak4[0,1] = trans_opt[0,1] + 2e-7

# to renormalize, we'll have to do the oposite to [0,0]
trans_tweak1[0,0] = trans_opt[0,0] + 2e-7
trans_tweak2[0,0] = trans_opt[0,0] + 1e-7
trans_tweak3[0,0] = trans_opt[0,0] - 1e-7
trans_tweak4[0,0] = trans_opt[0,0] - 2e-7

# make the models
model_tweak1 = h2mm_model(prior_opt,trans_tweak1,obs_opt)
model_tweak2 = h2mm_model(prior_opt,trans_tweak2,obs_opt)
model_tweak3 = h2mm_model(prior_opt,trans_tweak3,obs_opt)
model_tweak4 = h2mm_model(prior_opt,trans_tweak4,obs_opt)

#### <font color="green">Now run `H2MM_arr` (Error Evaluation)</font>

In [ ]:
# put the new models into a list
model_list = [model_tweak1, model_tweak2, opt_model_2state, model_tweak3, model_tweak4]
out_list = H2MM_arr(model_list,color,times)

#### Examine the results:
*NOTE: the cell bellow takes advantage of feature of Python called list comprehension, if you're confuse, check out this tutorial: it's a really powerful feature*.

In [ ]:
loglik = [model.loglik for model in out_list]
trans_rate = [model.trans[0,1] for model in out_list]
plt.scatter(trans_rate,loglik)

<a id='2'></a>
# 2. The `h2mm_model` object

<a id='21'></a>
## 2.1 Accessing core `h2mm_model` parameters

We've already seen how to look at the *loglikelihood* and $BIC$ from an optimized `h2mm_model` with `model.loglik` and `model.bic`. But you can also look at the arrays that make up the model- those all important **initial proability**, **transition probability** and **emission probability** matrices. They are named basically the same as how we defined them:
 - `.prior` for the **initial proability**
 - `.trans` for the **transition probability**
 - `.obs` for the **emission probability**

Whenever you call one of these properties, Python will return a NUMPY array. You can even assign it to a new variable, but note that if you do, changing the values in that variable will **not** change the values in the model, this is because under the hood, Python is actually copying the array in memory.

#### <font color="green">The initial model</font>

In [ ]:
prior_in = init_model_2state.prior
prior_in

In [ ]:
trans_in = init_model_2state.trans
trans_in

In [ ]:
obs_in = init_model_2state.obs
obs_in

#### <font color="green">The optimized model</font>

In [ ]:
prior_opt = opt_model_2state.prior
prior_opt

In [ ]:
trans_opt = opt_model_2state.trans
trans_opt

In [ ]:
obs_opt = opt_model_2state.obs
obs_opt

Now, we can change values in these arrays, or do whatever else you want. Since these are **copies** we will not be changing anything in the **models**.

One of the most obvious things to do is to convert the the **transition probability matrix** into units of $s^{-1}$, instead of whatever the clock of the measurement is.
*Remember that `H2MM_C` treats the arrival times as integers, so you have to know what the difference in time is between time 1 and time 2. In many nsALEX experiments, the file will store arrival times this way, and will include a clock rate, the time between 1 and 2 is so many nanoseconds.*

Let's choose a value common to nsALEX measurements: $50ns$, in seconds this is $5.0*10^{-5}s$

In [ ]:
# suppose the resoluiton of times is 50e-6 (50MHz), then dividing by that will give the transition rate in s^-1
clk_p = 5.0e-5
trans_opt = trans_opt/clk_p

print("trans_out = ")
print(trans_opt)
print("h_out.trans = ")
print(opt_model_2state.trans)

In [ ]:
if np.all(trans_opt == opt_model_2state.trans):
    print("Changing trans_out changed opt_model_2state.trans")
else:
    print("Changing trans_out left opt_model_2state.trans the same")

<a id='22'></a>
## <font color="gray">2.2 Review of `h2mm_model` Fields</font>

Now let's review what we've learnd about the `h2mm_model` fields:
1. Model Definition: these are the values that define a model:
    - `.prior` the **initial probability matrix** is a 1D array, and has length of the number of states in the model, the sum of all values in this matrix shoudl be 1, and none negative- it represents the probability of the system being in a given state at the beginning of a burst, so the system will be in a state at the beginning, and negative probabilities are meaningless)
    - `.trans` the **transition probability matrix** a 2D array, both dimensions being the number of states. Each row should sum to 1, as it represents the probability that if the system is in a state, what is the probability to transition to another state, so the system must tranistion to a state, even if it is to the same state (in fact staying in the same state is the most likely, which is why the diagonal of this matrix is usually close to 1)
    - `.obs` the **emission probability matrix** a 2D array with rows as the number of states, and columns the number of **streams** (see chapter 3, so far we have only dealt with 2 streams), the rows should sum to one, and none negative, as each row is the probability that if the system is in that state, it will emit a photon.
2. Likelihood Estimators:
    - `.loglik`the *loglikelihood* (log of likelihood), of the model, useful for comparing two models with the same number of states, run against the same set of data. *Initial models (before calculation of loglik against **data**) have a value of `-inf`*
    - `.bic` the Bayesian Information Criterion, an assement of the likelihood of the model, with penatlies for the number of states, allows comparison of models with different numbers of states, but both still run against the same set of data. *Initial models (before calculation of loglik against **data** will raise an error if this field is called.*

<a id='23'></a>
## 2.3 Changing Values in a Model

When you access the values in an `h2mm_model` object, you do not change them.

But what if you have a new set of data, or something else, and you would like to change those values? Well, that can be done by passing an array into the field: 

- `model.prior = new_prior`
- `model.trans = new_trans`
- `model.obs = new_obs`

But note that it must be the *same* size, otherwise an error will be raised.

However, it should be noted that the `H2MM_C` API automatically resets the `.loglik` to `-inf` and the `.bic` will raise a warning, when you set the value of one of these fields. This is good because the model is no longer the same, and therefore a new *loglikelihood* must be calculated.

#### <font color="green">Change the `.trans` property of an `h2mm_model` object:</font>

In [ ]:
# make a new trans array
trans_new = np.array([[0.9999999,1e-7],[2e-7,0.9999998]])
# assign to the model
opt_model_2state.trans = trans_new
# display the model
opt_model_2state

Note that the `.loglik` is now reset to `-inf`:

In [ ]:
opt_model_2state.loglik

So, we should re-optimize this model.

Since the model is no longer optimized, we should run `H2MM_arr` to get the new *loglikelihood.*

In [ ]:
opt_model_2state = H2MM_arr(opt_model_2state,color,times)
opt_model_2state

Now the *loglikelihood* is real:

In [ ]:
opt_model_2state.loglik

<a id='24'></a>
## 2.4 `h2mm_model` Methods

*To Matlab users: one of the biggest differences between Python and Matlab is that Python is "object oriented." I recomend this tutorial **OOP tutorial** to get used to the conventions. Basically this means you can execute functions on a variable with the dot ("`.`") notation, which ones depends on the type of the varaible, and these are generally called "methods"*

The `h2mm_model` object has a few methods that you might find useful, so let's go over them with some examples:

<a id='241'></a>
### 2.4.1 The `.copy()` method

In python, if we just execute `new_model = old_model` where `old_model` is a `h2mm_model` object, all we have really done is let ourselves refer to `old_model` in a new way. This means that if we change the `.prior`, `.trans` or `.obs` arrays in `new_model`, we will change those values in `old_model`.

#### <font color="green">Demonstrate Problem of using `new_model = old_model`</font>

In [ ]:
# get the current state of the trans model
old_trans = opt_model_2state.trans
# use the standard = operator without copy()
new_model = opt_model_2state
new_trans = np.array([[0.9999995,5e-7],[5e-7,0.9999995]])
new_model.trans = new_trans

# display the old model trans matrix, note that it changed
print("The current state of opt_model_2state.trans = ")
print(opt_model_2state.trans)
print("The old state of opt_model_2state.trans = ")
print(old_trans)
if np.all(old_trans == opt_model_2state.trans):
    print("The trans array is unchanged")
else:
    print("The trans array is changed")

The `H2MM_C` API includes the `.copy()` method, which returns a **copy** of the entire model, with its values stored separately in memory (for the experienced Python programmer, it makes a *deep copy* of the data).

In [ ]:
opt_model_2state = EM_H2MM_C(opt_model_2state,color,times)

#### <font color="green">Use the `.copy()` method</font>

In [ ]:
# get the current state of the trans model
old_trans = opt_model_2state.trans
# use the copy() method
new_model = opt_model_2state.copy()
new_trans = np.array([[0.9999995,5e-7],[5e-7,0.9999995]])
new_model.trans = new_trans

# display the old model trans matrix, note that it changed
print("The current state of opt_model_2state.trans = ")
print(opt_model_2state.trans)
print("The old state of opt_model_2state.trans = ")
print(old_trans)
if np.all(old_trans == opt_model_2state.trans):
    print("The trans array is unchanged")
else:
    print("The trans array is changed")

#### Practical example

Remember when we were evaluating the error, how we had to make a bunch of arrays and make new `h2mm_model` objects using them? Well, the `.copy()` method allows us to do this without having to extract all the arrays.

<a id='erroreval'></a>
#### <font color="blue">Error Evaluation with `.copy`</font>

In [ ]:
# make copies of the model for each tweaked model
model_tweak1 = opt_model_2state.copy()
model_tweak2 = opt_model_2state.copy()
model_tweak3 = opt_model_2state.copy()
model_tweak4 = opt_model_2state.copy()

# makes arrays for each tweak to adjust
trans_tweak1 = opt_model_2state.trans
trans_tweak2 = opt_model_2state.trans
trans_tweak3 = opt_model_2state.trans
trans_tweak4 = opt_model_2state.trans

# adjust the [0,1] element of the arrays
trans_tweak1[0,1] = trans_tweak1[0,1] - 2e-7
trans_tweak2[0,1] = trans_tweak2[0,1] - 1e-7
trans_tweak3[0,1] = trans_tweak3[0,1] + 1e-7
trans_tweak4[0,1] = trans_tweak4[0,1] + 2e-7

# adjust the [0,0] so that the arrays are row stochastic
trans_tweak1[0,0] = trans_tweak1[0,0] + 2e-7
trans_tweak2[0,0] = trans_tweak2[0,0] + 1e-7
trans_tweak3[0,0] = trans_tweak3[0,0] - 1e-7
trans_tweak4[0,0] = trans_tweak4[0,0] - 2e-7

# change the values in the trans arrays in the copies
model_tweak1.trans = trans_tweak1
model_tweak2.trans = trans_tweak2
model_tweak3.trans = trans_tweak3
model_tweak4.trans = trans_tweak4

# make a list out of the models
model_list = [model_tweak1, model_tweak2, opt_model_2state, model_tweak3, model_tweak4]
# evaluate the loglik for all the tweaked models
out_list = H2MM_arr(model_list,color,times)

loglik = [model.loglik for model in out_list]
trans_rate = [model.trans[0,1] for model in out_list]

plt.scatter(trans_rate,loglik)

In [ ]:
trans_rate, loglik

<a id='242'></a>
### 2.4.2 The `.evaluate` Method

Sometimes we'd like to just evaluate the *loglikelihood* of a given `h2mm_model` object against a set of **data**, and not have to make a separte object. We could use `model = H2MM_arr(model,color,time)`, but there is a cleaner way: the `.evaluate` method, which under the hood calls `H2MM_arr`, but  `.evaluate` is cleaner.

*For Matlab users: this will be a good example of how methods work in object oreiented programming*

The signature is basically the same: `model.evaluate(color,time)`

To demonstrate, let's make another model in our curve seeing how sharp the *loglikelihood* peak is around the transition rate.

#### <font color="gray">Make new tweaked model</font>

In [ ]:
# make the new trans array
trans_tweak5 = opt_model_2state.trans

trans_tweak5[0,1] = trans_tweak5[0,1] + 3e-7
trans_tweak5[0,0] = trans_tweak5[0,0] - 3e-7
# update the model
model_tweak5 = opt_model_2state.copy()
model_tweak5.trans = trans_tweak5

#### <font color="green">Get the *loglikelihood* of a model with `.evaluate`</font>

In [ ]:
model_tweak5.evaluate(color,times)
model_tweak4

Look at the `.loglik` specifically:

In [ ]:
model_tweak5.loglik

*This is for those coming from Matplotlib: let's add the new model to the list of models, and plot, this will demonstrate how Python lists and list comprehension works*

In [ ]:
out_list = out_list + [model_tweak5]

#### Now let's plot the updated list

In [ ]:
loglik = [model.loglik for model in out_list]
trans_rate = [model.trans[0,1] for model in out_list]
plt.scatter(trans_rate,loglik)

<a id='243'></a>
### 2.4.3 The `.optimize` Method

The `.optimize` does the same thing as `EM_H2MM_C` find the optimal model for some **data**, but just as a method of an `h2mm_model` object. The function call is `model.optimize(color,time)`. (very similar to `.evaluate`). 

Calling `model.optimize(color,time)` is the same as calling `model = EM_H2MM_C(model,color,time)`.

So let's try this, we'll make an unoptimized model, and then optimize it against our data using `.optimize` instead of `EM_H2MM_C`.

#### <font color="gray">Make an initial model</font>

In [ ]:
prior = np.array([0.5,0.5])
trans = np.array([[0.999994,6e-6],[4e-6,0.999996]])
obs = np.array([[0.8,0.2],[0.5,0.5]])
model_2state = h2mm_model(prior,trans,obs)

#### <font color="green">Optimize with `.optimize`</font>

In [ ]:
model_2state.optimize(color,times)
model_2state

<a id='244'></a>
### 2.4.4 `EM_H2MM_C` vs `.optimize` and `H2MM_arr` vs `.evaluate`: They're basically the same

The `h2mm_model` object method `.optimize` has basically the same function as the function `EM_H2MM_C`. In fact, the API uses `EM_H2MM_C` to evaluate `.optimize`. So what you can do with one, you can do with the other. Which one you use is mostly a matter of preference. 

The main difference is that when you use `EM_H2MM_C`, a new `h2mm_model` object is created, so it uses a **tiny** bit more memory, but you keep the old model around in case you want to use it again. On the other hand when you use `.optimize`, the results are returned to the original model, so you save a **tiny** amount of memory, but that original model is no longer available.

The choice between the `.evaluate` method and the `H2MM_arr` function is just slighly more complicated. The `.optimize` method calls `H2MM_arr` to evaluate the model, so `.evaluate` saves a little memory, but the old model is gone, while `H2MM_arr` gives you a new model, using a little more memory.

However, one more thing to be aware of is that the `H2MM_arr` function can also accept lists, tuples, and even NUMPY arrays of `h2mm_model` objects, and run them all against the same set of data. So it's a little more flexible in it's usage. Further, there is some preprocessing of the data that takes place before the main optimization. So if you have a lot of `h2mm_model` objects to evaluate, `H2MM_arr` will probably slightly faster, as this preprocessing takes once if you use `H2MM_arr`, while if you `.evaluate` each model, the preprocessing will be unnecessarily repeated each for each model.

<a id='3'></a>
# <font color="brown">3. Format of  Color and Time (Data)</font>

**This section is rather laborious and pedantic, many will be able to skip this section without much trouble**

OK, now that we have a basic idea of how the analysis works, let's get into the details of what everything is.

First, lets examine the **data**, i.e. the `color` and `times` lists (they can also be tuples).

The cell bellow that runs the Python function `type` on both to get the type of both

In [ ]:
type(color), type(times)

But we should check things a little further---

Think of these lists as matched pairs.
First, they have to be the same length, which we can check by calling the Python function `len`:

In [ ]:
len(color), len(times)

Now, let's look at just one burst:

In [ ]:
color[0]

In [ ]:
times[0]

See how the `colors[0]` is switching between 0 and 1, while all the elements of `times[0]`

Some things to note about bursts:
 - `colors[0]` and `times[0]` are NUMPY **integer** arrays, and all values are **positive** *these are even converted into unsigned integers in the code*
 - Values in `times[0]` are always increasing, this is because the represent the arrival times of the photons, so photons have to be "in order."
 - `colors[0]` as a seemingly random set of 0's and 1's. This is because they represent the index of the **stream** of each photon (i.e. 0 for Donor photons, and 1 for Acceptor photons(
 - The number of elements in `colors[0]` and `times[0]` is the same, because these are different aspects of the same photon

Now let's demonstrate that the same is true for all elements of the `color` and `times` lists, we'll use a nifty statement called `assert`, if it evaluates as false, then Python will raise an error and stop execution:

*For those coming from Matlab, the square brackets `[]` in Python indicate specifying an index, think of curly brackets and parenthesis `{}` and `()`*

In [ ]:
# loop over the length of the array
for i in range(0,len(color)):
    # the type of c and t (elements of color and times) is the same, and the same as the first element of each
    assert type(color[i]) == type(times[i]) == type(color[0]) == type(times[0])
    # show that the sizes of color[i] and times[i] are the same
    assert color[i].size == times[i].size
print("All arrays of same type and sizes of color/times match")

Note also that bursts do not have to be the same size:

In [ ]:
color[0].size, color[1].size

<a id='4'></a>
# 4. Multiparameter H<sup>2</sup>MM

<a id='41'></a>
## 4.1 Working with more than 2 streams

The previous data used only 2 **streams** of data, so all the values in the `color` arrays were either 0 or 1. However, in many experiments, there will be more **streams**-- photons can be distinguished in a number of ways. *(think donor vs acceptor excitation in addition to emission, or 3 color FRET, or even polarization)*

So how do we handle this? Well, prety simple, each new stream gets a new number (an index to be precise) in the `color` array. So if there are $n$ streams, they will be indexed succesively from $i=[0, 1, ..., n-1]$.

So let's load a data set that has 3 **streams** (this is the same code as before, just a different file, and stored in the variables `color3` and `times3`)

**For your data, it's your job to export it, if you're using *FRETBursts*, you should be able to copy-paste the `data_sort` function from the jupyter notebooks that go along with the mpH<sup>2</sup>MM Nature Communications paper.**

#### <font color="gray">Load 3 stream data</font>

In [ ]:
times3 = []
color3 = []
i = 0
# open the file
with open('sample_data_3det.txt') as f:
    for line in f: # this reads each line one at a time
        if i % 2 == 0: # the datafiel is formate
            times3.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color3.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1

#### <font color="brown">Let's inspect a color array to show that it has 3 **streams**</font>

In [ ]:
color3[0]

See how instead of being a "random" set of 0's and 1's, it's now a "random" set of 0's, 1's and 2's? That means it's a 3 **stream** model.

<a id='42'></a>
## 4.2 Setting up the **emission probability matrix**

With 3 **streams** the **emission probability** matrix has 3 columns, instead of the 2 that it had with the previous 2 **stream** data. Simple coorespondence. Thankfully **initial probability** and **transition probability** matrices are *unchanged*.

To demonstrate that only **emmission probability** matrix changes, we'll use the **initial probability** and **transition probability** matrices we defined all the way back in secion 1.4 (so make sure you've run those cells, otherwise this next cell won't work)

#### <font color="green">Define some new `.obs` arrays, and make 3 stream models</font>

In [ ]:
# define the obs arrays
obs_1state3 = np.array([[0.3,0.3,0.4]])
obs_2state3 = np.array([[0.5,0.1,0.4],[0.1,0.5,0.4]])
obs_3state3 = np.array([[0.4,0.4,0.2],[0.3,0.1,0.6],[0.2,0.4,0.4]])
obs_4state3 = np.array([[0.4,0.4,0.2],[0.3,0.1,0.6],[0.2,0.4,0.4],[0.1,0.1,0.8]])

# make the models
init_model_1state3 = h2mm_model(prior_1state,trans_1state,obs_1state3)
init_model_2state3 = h2mm_model(prior_2state,trans_2state,obs_2state3)
init_model_3state3 = h2mm_model(prior_3state,trans_3state,obs_3state3)
init_model_4state3 = h2mm_model(prior_4state,trans_4state,obs_4state3)

#### Run the optimization

Note that other than making sure the number of **streams** in the data matches the number of columns in the **emission probability** matrix, optimization and *Viterbi* work exaclty the same.

In [ ]:
opt_model_1state3 = EM_H2MM_C(init_model_1state3,color3,times3)
opt_model_2state3 = EM_H2MM_C(init_model_2state3,color3,times3)
opt_model_3state3 = EM_H2MM_C(init_model_3state3,color3,times3)
opt_model_4state3 = EM_H2MM_C(init_model_4state3,color3,times3)

#### Run *Viterbi* to get the $ICL$

In [ ]:
path_1state3, scale_1state3, loglik_1state3, icl_1state3 = viterbi_path(opt_model_1state3,color3,times3)
path_2state3, scale_2state3, loglik_2state3, icl_2state3 = viterbi_path(opt_model_2state3,color3,times3)
path_3state3, scale_3state3, loglik_3state3, icl_3state3 = viterbi_path(opt_model_3state3,color3,times3)
path_4state3, scale_4state3, loglik_4state3, icl_4state3 = viterbi_path(opt_model_1state3,color3,times3)

#### Plot $BIC$

In [ ]:
opt_model_list3 = [opt_model_1state3,opt_model_2state3,opt_model_3state3,opt_model_4state3]
bic3 = [model.bic for model in opt_model_list3]
plt.scatter([1,2,3,4],bic3)
plt.title("BIC")

#### Plot $ICL$

In [ ]:
icl3 = [icl_1state3,icl_2state3,icl_3state3,icl_4state3]
plt.scatter([1,2,3,4],icl3)

So, it looks like the 3 state model is best for this set of data.

<a id='43'></a>
## 4.3 Checking a Model: `.ndet` and `.nstate` fields in `h2mm_model` objects

Since `h2mm_model` objects have a number of states based on the size of all their matrices, as well as a number of **streams**, based on the number of columns in their **emission probability** matrix (`.obs`). 

We could check the number of states by checking the shape of `.prior` or `.trans`, even `.obs`, and we could check the number of detectors by checking the shape of `.obs`

So let's do that, and then we'll introduce a better method:

In [ ]:
opt_model_2state3.trans.shape

So this model has `.trans` of shape 2x2, so it is a 2 state model.

We could find this with `opt_model_2state3.prior.shape` as well.

If we look at the `.obs` field, we can get the number of **streams** as well:

In [ ]:
opt_model_2state3.obs.shape

So the shape is `(2, 3)` which means that it is a 2 state model for 3 **streams**.

So, how about the better way?

The `h2mm_model` object has special fields for these: `.nstate` and `.ndet` for the number of **states** and **streams**.
#### <font color="green">Use `.nstate` and `.ndet` fields</font>

In [ ]:
opt_model_2state3.nstate, opt_model_2state3.ndet

These fields cannot be set, once a model is made, the number of **states** (`.nstate`) and **streams** is fixed (`.ndet`). These fields let us access these values in a streamlined way, which can be useful to make sure that a model has the appropriate number of streams for the data.

<a id='5'></a>
# 5. Controlling `EM_H2MM_C`

Sometimes we may want to restrain what values are possible in the H<sup>2</sup>MM model, for instance to keep transition rates within reasonable values, or because you know something about how the emmission probability matrix (`obs`) should behave. (This feature was initially designed for $\mu$sALEX measurements)

<a id='51'></a>
## 5.1 Convergence

At it's core, **H<sup>2</sup>MM** is an optimization. Further, the mathematics and algorithm is formulated such that whenever the loglikelihood of a model is calculated, a new model is also generated, which is mathematically guaranteed to have a better *loglikelihood* than the previous model, unless the maximum has been reached. (The model is said then to have converged).

Two (and sometimes related) issues arrise:
1. Floating point errors 
    - The term "maximum" needs a little wiggle room, the models with differences in *loglikelihood* smaller than the floating point error should be considered as of equal quality.
2. Long optimization
    - Coming to the "maximum" *loglikelihood* takes a very large number of iterations, and usually the differences both in values and *loglikelihoods* are minor.
     - This issue most often happens for large numbers of states, and overfit models.

The solution comes by setting a maximum number of iterations the optimization will proceed, and setting a small, but non-zero threshold for the difference between *loglikelihoods* at which to consider sufficient to consider the optimization to have converged.

By default, `EM_H2MM_C` will stop optimizing after 3600 iterations, and considers a difference in *loglikelihood* of $10^{-14}$ to be small enough to consider the optimization to have converged.

These values can be overiden with the `max_iter` and `converged_min` keyword arguments (aka kwargs). 

*For Matlab users: keyword arguments are optional arguments with a default value. These are specified inside the parenthesis after a function, after all the normal arguments, and specified as `kwarg=value`*

#### <font color="green">Changing `max_iter`</font>

In [ ]:
small_opt_model_3state = EM_H2MM_C(init_model_3state, color, times, max_iter=500)

#### <font color="green">Changing `converged_min`</font>

In [ ]:
large_conv_model_3state = EM_H2MM_C(init_model_3state, color, times, converged_min=4e-6)

There is a final keyword argument that can help control the length of optimization: `max_time`. It does as advertized, if the optimization goes over the time (which is in seconds), the optimization will stop at whatever model is currently being calculated.

<font color="brown">**HOWEVER** `max_time` *uses an **innacurate clock**, which tends to run **fast** so it's use is not recommended*</font>

By default `max_time` is infinity, and so a model will never time out.

<a id='511'></a>
### 5.1.1 Using `.optimize` instead

The `.optimize` method takes the same keyword arguments as `EM_H2MM_C`, in the cell below, try using them to prove it to yourself.

In [ ]:
# make a new model for yourself to use as a guinea pig
play_model = h2mm_model(prior_3state,trans_3state,obs_3state3)
# perform the optimization
play_model.optimize(color3,times3,max_iter=800)

<a id='52'></a>
## 5.2 Limits

In some data sets, there is the potential that we will want to exclude certain values, or otherwise restrain the fitting to only include certain ranges of values for various parameters. 

<font color="brown">This is expected to happen most often (but not exclusively) when there is some experimental parameter is periodic, but not important to the data. For instance in $\mu$sALEX experiments, the laser alternation period causes donor and acceptor excitation photons, which arrive in separate streams to alternate (and thus transition) with the definition of periodicity, yet that has no bearing on transitions between our system. Thus transition rates close to the rate of laser alternation are likely to be artifacts, and thus we want to exclude them. For $\mu$sALEX experiments, we find this still is not enough, </font> 

<a id='521'></a>
### 5.2.1 Our First Bounded Fit

To define some bounds, we need to define the bounds, this is done using the `h2mm_limits` object, which we pass into the `EM_H2MM_C` function through the keyword argument `bounds`, and we also need to supply a method string to another keyword argument, `bounds_func`.

Let's see an example, and then explain:
#### <font color="green">Make an `h2mm_limits` object</font>

In [ ]:
alt_period = 4000 # a fake alternation period
us_bounds = h2mm_limits(max_trans = 1/(alt_period))

#### <font color="green">Run Optimization with bounds</font>

In [ ]:
# run optimization with bounds
us_opt_model3 = EM_H2MM_C(init_model_3state3,color3,times3, bounds_func='revert',bounds=us_bounds)
us_opt_model3

So, what did we just do? The `h2mm_limits` object `us_bounds` prevents any value (off the diagonal) of the **transition probability** matrix (`.trans`) from ever being larger (ie faster transition rate) than `1/(4000)`. 

#### Bounds process
When you use a bounds method, each iteration goes through the following steps:
1. Calculate *loglikelihood* and new model
2. Check if model converged
3. Analyze New model, and correct if necessary
    1. Check if any values are smaller or larger than a pre-set minimum or maximum
    2. If values are out of bounds, apply correction, method defined by argument passed to `bounds_func`
4. Repeat optimization (back to step 1)

<a id='522'></a>
### 5.2.2 The `h2mm_limits` object

`h2mm_limits` objects are designed specifically to be a flexible container for defining minimum and maximum values for elements of any and/or all of the `.prior`, `.trans`, and `.obs` arrays in an `h2mm_model` object during optimization.

Notice that all arguments passed to `h2mm_limits()` are **keyword arguments**. Names are fairly obvious, with a `min_` or `max_` placed before the array name to define the minimum or maximum values respectively, so we have:
- `min_prior` and `max_prior` to bound the `.prior` array
- `min_trans` and `max_trans` to bound the `.trans` array
- `min_obs` and `max_obs` to bound the `.obs` array

These can be specified either as single floats (decimal number), or as arrays.
Specifying as a float will mean that:
1. All values will be bounded by the same minimum or maximum
    - For the `.trans` array, the values on the diagonal are ignored, as these will nearly always be close to 1
2. The same `h2mm_limits`object can be used with `h2mm_model` objects with different **states** and **streams**

Specifying as an array will mean that:
1. Each element of the given array will be bounded separately, allowing greater precisison
    - `.trans` diagonals are ignored
2. The `h2mm_limits` object will only be able to be used with certain `h2mm_model` objects.
    - If any of the following `min_prior`, `max_prior`, `min_trans` or `max_trans` is specified as an array, then the `h2mm_limits` object must be used top optimize `h2mm_model` objects that have the same number of states
    - If `min_obs` or `max_obs` is specified, then the `h2mm_limits` object can only be used to optimize `h2mm_model` objects with the same number of **states** and **streams**

In the previous example, we specified `max_trans` as a float. Therefore, it can be used to bound just about any model:

#### Bounding several different models, with different numbers of streams and states:

In [ ]:
us_opt_model_1state = EM_H2MM_C(init_model_1state,color,times, bounds_func='minmax',bounds=us_bounds)
us_opt_model_2state = EM_H2MM_C(init_model_2state,color,times, bounds_func='minmax',bounds=us_bounds)
us_opt_model_3state = EM_H2MM_C(init_model_3state,color,times, bounds_func='minmax',bounds=us_bounds)
us_opt_model_1state3 = EM_H2MM_C(init_model_1state3,color3,times3, bounds_func='minmax',bounds=us_bounds)
us_opt_model_2state3 = EM_H2MM_C(init_model_2state3,color3,times3, bounds_func='minmax',bounds=us_bounds)
us_opt_model_3state3 = EM_H2MM_C(init_model_3state3,color3,times3, bounds_func='minmax',bounds=us_bounds)

#### But, if we specify a different parameter with an array, then only matching models can be used:
This time, let's work with the `min_obs` and `max_obs` arrays. This will restrict the individual states of the `.obs` matrix vary within certain ranges, each state to a different area. 
<font color="brown">*This could be used for instance if we want to have only one state in a given area of the multi-dimensional space of the **emission probability** matrix, and all the other states in some other area of that range.*</font>

In [ ]:
# define minimum and maximum arrays for obs, this will be a 4 state 3 detector model
obs_min = np.array([[0.0,0.0,0.0],[0.1,0.0,0.3],[0.0,0.1,0.4],[0.0,0.0,0.7]])
obs_max = np.array([[1.0,0.5,1.0],[0.6,0.3,0.7],[0.6,0.8,0.8],[1.0,1.0,1.0]])
# make the limit object
obs_limit = h2mm_limits(min_obs=obs_min, max_obs=obs_max)

#### Run the optimization

In [ ]:
obs_opt_model_4state3 = EM_H2MM_C(init_model_4state3,color3,times3)
obs_opt_model_4state3

<a id='523'></a>
### 5.2.3 `bounds_func` Options

There are several methods for "correcting" a new model that has "out of bounds" values. We can choose which one to use with the `bounds_func` keyword argument. These are all identified with strings.

Our options are:
- `'minmax'` when a value is out of bounds, it is set to the minimum or maximum depending on which was exceeded
- `revert` resets the value to what it was in the current model (the one whose *loglikelihood* was just calculated
- `revert_old` resets the value to the value in the model that preceeded the current one in optimization

<font color="brown">How to choose between these three functions? It depends on what we want to limit/achieve. `'minmax'` will tend to just keep the values at their minimums/maximums, so it's great if we just don't want to optimzie for really slow transition rates that are longer than our experiment, and thus never happen and therefore their values are more likely statistical noise. `'revert'` on the other hand will be more likely to be "pushed off" the path it was going and find a local maxima in the *loglikelihood*, away from artifacts produced by some experimental periodicity. But it also carries the greater risk of pre-mature convergence. `'revert_old'` is basically just more extreme, even more likely to find a local maxima in *loglikelihood*, but also more likely to converge pre-maturely.</font>

#### <font color="green">Using an alternative `bounds_func` option</font>
*Try switching it between the different options

In [ ]:
usminmax_opt_model_2state3 = EM_H2MM_C(init_model_2state3,color3,times3, bounds_func='revert',bounds=us_bounds)
usminmax_opt_model_2state3

Finally, be aware that `h2mm_limits` is very flexible, any combination of min/max limits can be specified, and it can take mixing arrays and floats. The one limitation is that arrays must have matching **states** and **streams**. If conflicting arrays are given, an error will be raised, halting the Python script, and the conflict will need to be corrected. The `H2MM_C` API also checks for other potential conflicts in the value provided, i.e. a minimum cannot have a value greater than a maximum.

#### A `h2mm_limits` object with multiple types of array specified mixing array and float definitions:

In [ ]:
comb_bounds = h2mm_limits(max_trans=1/alt_period, min_obs=obs_min,max_obs=0.99999)
comb_opt_model_2state3 = EM_H2MM_C(init_model_4state3,color3,times3, bounds_func='revert',bounds=comb_bounds)

<a id='53'></a>
## 5.3 Auto-generate Initial Model with `factory_h2mm_model` function

In [section 1.4](#wholeanalysis1), we had to define with all the arrays a new `h2mm_model` object for each number of **states** we wanted to optimize. This meant that if the system has 5 states, we'd have to define a 5<sup>th</sup> , and probably even a 6<sup>th</sup> model to be sure that 5 states is the best fit.

It is better to define initial models based on prior knowledge of the system, but especially when we have a large number of states, or are trying to fit several different data sets, this can be rather laborious.

The `H2MM_C` API contains a function to help us: `factory_h2mm_model`, which returns an initial `h2mm_model` object.

The basic signature is: `factory_h2mm_model(nstate,ndet)` where `nstate` is the number of **states** in the output model, and `ndet` is the number of **streams** in the output model.

<a id='531'></a>
### <font color="green">5.3.1 Use `factory_h2mm_model` to make initial model</a>
Try changing the arguments, and see how it changes the **streams** and **states** of the model.

In [ ]:
f_init_model_3state3 = factory_h2mm_model(3,3)
f_init_model_3state3

So let's re-write the code in [Section 1.4](#wholeanalysis1) using the `factory_h2mm_model`function:

<a id='fhmwrkflow'></a>
#### <font color="blue">Basic Analysis with `factory_h2mm_model` to generate models</font>

In [ ]:
# load data (this is the same as before)
times = []
color = []
i = 0
# open the file
with open('sample_data_2det.txt') as f:
    for line in f: # this reads each line one at a time
        if i % 2 == 0: # the datafiel is formate
            times.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1


# Define models using factory_h2mm_model
finit_model_1state = factory_h2mm_model(1,2)
finit_model_2state = factory_h2mm_model(2,2)
finit_model_3state = factory_h2mm_model(3,2)
finit_model_4state = factory_h2mm_model(4,2)


# Optimize each model
fopt_model_1state = EM_H2MM_C(finit_model_1state,color,times)
fopt_model_2state = EM_H2MM_C(finit_model_2state,color,times)
fopt_model_3state = EM_H2MM_C(finit_model_3state,color,times)
fopt_model_4state = EM_H2MM_C(finit_model_4state,color,times)


# calculate ICL
path_1state, scale_1state, ll_1state, icl_1state = viterbi_path(fopt_model_1state,color,times)
path_2state, scale_2state, ll_2state, icl_2state = viterbi_path(fopt_model_2state,color,times)
path_3state, scale_3state, ll_3state, icl_3state = viterbi_path(fopt_model_3state,color,times)
path_4state, scale_4state, ll_4state, icl_4state = viterbi_path(fopt_model_4state,color,times)

<a id='532'></a>
### 5.3.2 Bounds and `factory_h2mm_model`

The `facotry_h2mm_model` function however can create problems when we are also bounding the values with `h2mm_limits` objects ([section 5.2](#52)). As it might generate a model with values that are out of the defined bounds. This will result in `EM_H2MM_C` potentially behaving oddly. This issue can be avoided by supplying the `h2mm_limits` object to `factory_h2mm_model` through the keyword argument `bounds`.

#### <font color="green">Factory model with bounds</font>

In [ ]:
us_bounds = h2mm_limits(max_trans = 1/4000)
binit_model_3state3 = factory_h2mm_model(3,3,bounds=us_bounds)
binit_model_3state3

**The `h2mm_limits` object used for bounding should always be supplied to `factory_h2mm_model`**

<a id='533'></a>
### 5.3.3 Distribution kwargs in `factory_h2mm_model`

There are a few additional keyword arguments to control how `factory_h2mm_model` distributes the values in the matrices of the output `h2mm_model` object.

These are: 
- `prior_dist` can be `'even'` or `'random'`
- `trans_dist` can be `'even'` or `'random'`
- `obs_dist` can be `'even'` or `'random'`

which set how values are distributed in the `.prior`, `.trans` and `.obs` arrays. There are only two possible inputs to these: `'even'` and `'random'` (both stings).

When the value is set to `'even'` (the default), then values are evenly distributed inside the respective arrays.

When `'random'` is chosen on the other hand, values will be assinged randomly in the respective arrays.

<font color="brown">`'even'` is usually a better choice, as the output is predictable, and an even distribution is generally a better starting point for optimization. `'random'` is useful for generating multiple initial **models** of the same **states** and **streams** to check if optimization finds the local or global maximum for the data. Further, occasionally when the `min_obs` or `max_obs` of the `bounds` are specified as arrays, setting `obs_dist` to `'random'` may be beneficial</font>

A final keyword argument, `trans_scale` is a float, and sets the off-diagonal values of the **transition probability** matrix (`.trans`). If `trans_dist = 'even'` and the default value does not conflict with the `bounds` input, then all off-diagonal values will be set to `trans_scale`, if `trans_dist = 'random'` then the values will be random, but centered around `trans_scale`.

With all this information, it is now possible to re-write our analysis to limit the model tranisition rates for $\mu$sALEX measurements, while also using the `factory_h2mm_model` function:

<a id='534'></a>
### <font color="blue">5.3.4 Full analsis with bounds and `factory_h2mm_model`</font>

In [ ]:
# load data (this is the same as before)
times = []
color = []
i = 0
# open the file
with open('sample_data_3det.txt') as f:
    for line in f: # this reads each line one at a time
        if i % 2 == 0: # the datafiel is formate
            times3.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color3.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1


# Define bounds object
alt_period = 4000
us_bounds = h2mm_limits(max_trans=1/alt_period)


# Define models using factory_h2mm_model
finit_model_1state3 = factory_h2mm_model(1,3,bounds=us_bounds)
finit_model_2state3 = factory_h2mm_model(2,3,bounds=us_bounds)
finit_model_3state3 = factory_h2mm_model(3,3,bounds=us_bounds)
finit_model_4state3 = factory_h2mm_model(4,3,bounds=us_bounds)


# Optimize each model
fopt_model_1state3 = EM_H2MM_C(finit_model_1state3,color3,times3,bounds=us_bounds,bounds_func='revert')
fopt_model_2state3 = EM_H2MM_C(finit_model_2state3,color3,times3,bounds=us_bounds,bounds_func='revert')
fopt_model_3state3 = EM_H2MM_C(finit_model_3state3,color3,times3,bounds=us_bounds,bounds_func='revert')
fopt_model_4state3 = EM_H2MM_C(finit_model_4state3,color3,times3,bounds=us_bounds,bounds_func='revert')


# calculate ICL
path_1state3, scale_1state3, ll_1state3, icl_1state3 = viterbi_path(fopt_model_1state3,color3,times3)
path_2state3, scale_2state3, ll_2state3, icl_2state3 = viterbi_path(fopt_model_2state3,color3,times3)
path_3state3, scale_3state3, ll_3state3, icl_3state3 = viterbi_path(fopt_model_3state3,color3,times3)
path_4state3, scale_4state3, ll_4state3, icl_4state3 = viterbi_path(fopt_model_4state3,color3,times3)

plt.scatter([1,2,3,4],[icl_1state3,icl_2state3,icl_3state3,icl_4state3])

<a id='54'></a>
## 5.4 "Continuous" Analysis

In the above example, all optimizations were run together, then all *Viterbi* analyss This is more clear and easier to read, however, it makes it harder to add more **states** to the analysis, so it is often more convinient to optimize, perform *Viterbi*, and then decide whether to optimize with one more **state**, or if we are confident that we have found the optimal number of **states** (usually by seeing that one or more models are over-fit).

So bellow is an example of such code:

<a id='optwrkflow'></a>
### <font color="blue">Optimize-> *Viterbi* workflow:</font>

In [ ]:
# load data (this is the same as before)
times = []
color = []
i = 0
# open the file
with open('sample_data_3det.txt') as f:
    for line in f: # this reads each line one at a time
        if i % 2 == 0: # the datafiel is formate
            times3.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color3.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1


# Define bounds object
alt_period = 4000
us_bounds = h2mm_limits(max_trans=1/alt_period)


# 1 state
finit_model_1state3 = factory_h2mm_model(1,3,bounds=us_bounds)
fopt_model_1state3 = EM_H2MM_C(finit_model_1state3,color3,times3,bounds=us_bounds,bounds_func='revert')
path_1state3, scale_1state3, ll_1state3, icl_1state3 = viterbi_path(fopt_model_1state3,color3,times3)

# 2 state
finit_model_2state3 = factory_h2mm_model(2,3,bounds=us_bounds)
fopt_model_2state3 = EM_H2MM_C(finit_model_2state3,color3,times3,bounds=us_bounds,bounds_func='revert')
path_2state3, scale_2state3, ll_2state3, icl_2state3 = viterbi_path(fopt_model_2state3,color3,times3)

# 3 state
finit_model_3state3 = factory_h2mm_model(3,3,bounds=us_bounds)
fopt_model_3state3 = EM_H2MM_C(finit_model_3state3,color3,times3,bounds=us_bounds,bounds_func='revert')
path_3state3, scale_3state3, ll_3state3, icl_3state3 = viterbi_path(fopt_model_3state3,color3,times3)

# 3 state
finit_model_4state3 = factory_h2mm_model(4,3,bounds=us_bounds)
fopt_model_4state3 = EM_H2MM_C(finit_model_4state3,color3,times3,bounds=us_bounds,bounds_func='revert')
path_4state3, scale_4state3, ll_4state3, icl_4state3 = viterbi_path(fopt_model_4state3,color3,times3)

plt.scatter([1,2,3,4],[icl_1state3,icl_2state3,icl_3state3,icl_4state3])

<a id='55'></a>
## <font color="blue">5.5 While loop analysis</font>

The following code is a full analysis, using all the main techniques we have learned. It is "smart" in analyzing the fitting results so that it only calculates enough states to be *relatively* confident that we have found the ideal model.

The basic algorithm is based on the following loop:
1. Create model with N states
2. Optimize model
3. Calcualte $ICL$ with *Viterbi*
4. Compare all calculated models to test if the optimial model has been found
    1. Should have at least 1 model with more states than optimal $ICL$
    2. Should have mininum number of states calculated
    3. Set a maxium number of states to calculate- as a way to prevent over-long analysis

The following code also incorporates `h2mm_limits` and `factory_h2mm_model`.
In practice, it is prefered to define 1 to 4 or 5 state models manually, and then switch to `factory_h2mm_model` to define larger models. Note that now results are stored in lists

<a id='finaldemo'></a>
#### <font color="blue">Full loop-based analysis</font>

In [ ]:
# Load data file
times3 = []
color3 = []
with open('sample_data_3det.txt') as f:
    for i, line in enumerate(f): # this reads each line one at a time
        if i % 2 == 0: # the datafiel is formate
            times3.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color3.append(np.array([int(x) for x in line.split()],dtype='L'))

# make bounds object
alt_period = 4000
us_bounds = h2mm_limits(max_trans=1/alt_period)

# setup lists for storing results
init_model = []
opt_model = []
path = []
scale = []
ll = []
icl = []
n = 1 # tracks number of states the model in the current iteration of the while loop should have
min_states = 4 # minimum number of states thas must be calculated
max_states = 8 # number of states beyond which to stop optimizing
cont = True

while cont:
    # make the n state model
    init_model_curr = factory_h2mm_model(n,3,bounds=us_bounds)
    init_model.append(init_model_curr) # append to list so we can inspect it later
    # optimize
    opt_model_curr = EM_H2MM_C(init_model_curr,color3,times3,bounds=us_bounds,bounds_func='revert')
    opt_model.append(opt_model_curr) # append to list so it can be examined at the end of the anlaysis
    # Viterbi
    path_curr, scale_curr, ll_curr, icl_curr = viterbi_path(opt_model_curr,color3,times3)
    # append results so they can be inspected later
    path.append(path_curr)
    scale.append(scale_curr)
    ll.append(ll_curr)
    icl.append(icl_curr)
    # Evaluate if we have sufficient knowledge to know if we have found the best fit model
    if n < min_states: # minimum number of models to optimize
        cont = True
        n += 1 # add 1 to n for next number of states to be calculated
    elif n > max_states: # don't optimize beyond some number of models
        cont = False
    elif np.min(icl) == icl_curr: # this says that the current icl is the minimum, therefore we do not have at least 1 overfit model
        cont = True
        n += 1 # add 1 to n for next number of states to be calculated
    else: # we have at least 1 overfit model, therefore we can stop optimizing
        cont = False

# evaluating the results
best_model_index = np.argmin(icl) # get index of lists of the "best" model, ie the one with the minimu ICL

# plot the ICL graph
plt.scatter(np.linspace(1,n,n),icl)
opt_model[best_model_index] # final display the "best model"

<a id='6'></a>
# 6. Customizing `EM_H2MM_C`

The `EM_H2MM_C` function (and the `.optimize` method) has many options and ways to customize it's function. These are all specified through keyword arguments. So let's go through them.

<a id='61'></a>
## 6.1 Changing parallelization: the `num_cores` kwarg

The `EM_H2MM_C` algorithm is parallelized so it can use multiple cores on the computer <font color="teal">(for the experience Pythnon programer, it ignores the gil)</font>. By default, the algorithm asks Python for the number of cores the machine has, and will limit itself to use only that many cores. But, for whatever reason, if we want to change this number, we have the keyword argument `num_cores`. 

<font color="brown">Why would we want to change `num_cores`? Maybe we have a  leave a core or two open for other things because we have some other task running. Further, each system is a little different, and while the default should be close to optimum, we might be able to eek out a tiny bit better performance by changing `num_cores`</font>

Let's pretent we want to do a bunch of other things in other programs while `EM_H2MM_C` optimizes a model, so we'll use just one core, leaving the others free for our precious other programs. To do this, set `num_cores=1`

#### <font color="green">Optimization using only one core</font>

In [ ]:
onecore_model_3state3 = EM_H2MM_C(init_model_3state3,color3,times3,num_cores=1)
onecore_model_3state3

Try playing with the number of cores, and with different initial models, and see how it affects the time.

*NOTE: changing the number of cores may cause the nubmer of iterations before it converges, this is due to floating point errors. Check the actual results, and values of a 1 core vs n core optimization will differ by less than $10^{-10}$, the same can cause variation from optimization to optimization if `converged_min` is the default, or smaller.*

<font color="brown">*In Jupyter notebooks, there are so-called "magic" commands, which start with % if applied to a single line, and %% to apply to an entire cell. The %%timeit magic can tell you how long it takes to execute a certain command, so it is included in the cell below to let you measure the time it takes for optimizations to run.*</font> 

In [ ]:
%%timeit
onecore_model_3state3 = EM_H2MM_C(init_model_3state3,color3,times3,num_cores=1)

`num_cores` is also a keyword argument in `viterbi_path`, `viterbi_sort`, and `H2MM_arr`. In all cases the underlying operation is the same.

<font color="teal">For the Python programmer: The default value for `num_cores = os.cpu_count()//2` The `//2` is because most systems are hyperthreaded, especially the higher powered systems researchers are likely to be working on. Therefore `os.cpu_count()` will return twice as many physical cores as there really are. If your system is not hyperthreaded, then you will want to manually specify `num_cores`. But I've found that depending on which OS/distro I'm using, how exactly it handles the parallelization differes. Therefore some experimenting with `num_cores` is likely to gain you a little speed up over the default.</font>

<font color="magenta">For the **C** programmer: All parallelizatin is handeled at the **C** level. For Linux, `H2MM_C` uses pthreads, while in Windows, the CreateThread function is used. The value passed to `num_cores` is the number of threads the algorithm will use. To Python programers, understand that in **C**, threads behave much more like Python processes, in that they can run simultameously on multiple cores.</font>

<a id='62'></a>
## 6.2 Printing `EM_H2MM_C` iterations

So, what if we want to track the progress of an optimization? If you've been paying close attention to the console/terminal window, you will have noticed that 

```
Iteration 0 ...
Iteration 1 ...
Iteration 2 ...
...
```

gets printed to this window whenever `EM_H2MM_C` is running. This helps keep track of the optimization, especially for ones with large numbers of states, which take longer, without 

<a id='621'></a>
### 6.2.1 Basic `print_func` options

- `'console'` the default, prints to the **consle** window, provides basic information on the progress of the optimization 
- `'all'` prints a representation of the whole model to be optimized in the next iteration (this will be very verbose <font color="brown">This option is very verbose, use only if you want to see the exact steps of the optimization</font>
- `'diff'` Print the difference between the previous and current model logliklihoods and the current loglilihood (this is the default if no `bounds` keyword argument is specified)
- `'comp'` Print the old and current logliklihoods
- `'iter'` Prints only the iteration number- compact way to still track optimization progress in Jupyter Notebook
- `None` Supresses all printing of iteration information, to both **console** and **jupyter notebook**

*If iterations show up in chuncks when printing to a jupyter notebook, this is because the display to the Jupyter notebook is routed through a buffer, so what is printed to the notebook is often a few milliseconds behind, and multple buffered messages get stored at once*

### Below you have an example, try changing it to the different options and see how it changes the output

In [ ]:
print_opt_model_2state3 = EM_H2MM_C(init_model_2state3,color3,times3,print_func='diff')

<a id='622'></a>
### 6.2.2 Customized display using your own function

There is one final type of argument that can be passed to `print_func`: a user-written function. When `EM_H2MM_C` detects that a function is passed to `print_func`, it will pass a specific set of arguments to that function:

`print_func(niter, new_model, current_model, old_model, t_iter, t_total)`

NOTE: It is not necessary, but recomended to keep these variable names in the function declaration.

where:
- `niter` is the number of iterations
- `new_model` is a `h2mm_model` object that represents the next model to be optimized (**before** checking for out of bounds values) note that its `.loglik` will be irrellevant because it has not been calculated yet.
- `current_model` is a `h2mm_model` object that represents the model whose `.loglik` was just calculated
- `old_model` is a `h2mm_model` object that represents the model from the previous iteration.
- `t_iter` is a float which is the time in seconds based on the **inaccurate C clock** that it took to calculate the latest iteration
- `t_total` is a float which is the time in seconds based on the **inaccurate C clock** that the full optimization has taken

We must write a function that accepts these arguments, and prints out whatever information we want to be displayed.

NOTE: This function should be short, just whatever pre-processing is needed and then a `print` statement.

#### Write a function to print the BIC of each iteration

In [ ]:
def bic_print(niter, new_model, current_model, old_model, t_iter, t_total):
    # the function signature needs to have the given order, but the names don't matter
    # however for readability, it's best just to keep the function signature exactly as is
    #print(f"The current BIC is {current.bic}, and we've been going for a full {t_total} seconds")
    print(f"Iteration:{niter}, the BIC is {current_model.bic}, iteration time {t_iter} seconds")
    # also, these functions will usually be really short, like this one, just a single print statement

Now, to pass this function, we just need to pass it to the `print_func` keyword argument **without** paranthesis.

<font color="brown">*For Matlab users, when Python sees a function name without parentheses, it does not try to execute it, rather it tells Python to pass that function along, and it can be called later by the function it was passed to.*</font>

#### <font color="green">Pass the `bic_print` function to `print_func` keyword argument</font>

In [ ]:
bic_print_opt_model_2state3 = EM_H2MM_C(init_model_2state3,color3,times3,print_func=bic_print)

<a id='63'></a>
## 6.3 User Defined Limits Function

When I said that the `bounds` keyword argument had to be `None` or a `h2mm_limits` object, and `bounds_func` had to be one of `'minmax'`, `'revert'`, or `'revert_old'`, well there is one exception: a function can be passed to `bounds_func`.

So let's first review how the optimization works:
When you use a bounds method, each iteration goes through the following steps:
1. Calculate *loglikelihood* and new model
2. Check if model converged
3. Analyze New model, and correct if necessary
    1. Check if any values are smaller or larger than a pre-set minimum or maximum
    2. If values are out of bounds, apply correction, method defined by argument passed to `bounds_func`
4. Repeat optimization (back to step 1)

When `EM_H2MM_C` detects that a function has been passed to `bounds_func`, then it hands over step 3 to the function passed to `bounds_func`.

This function must have the following signature:

`bounds_func(new_model,current_model,old_model,bound)`

where
- `new_model` is a `h2mm_model` object representing the unoptimized model to be calculated in the next iteration
- `current_model` is a `h2mm_model` object representing the model of the current iteration, whose `.loglik` was just calculated
- `old_model` is a `h2mm_model` object representing the model from the previous iteration
- `bound` is the argument passed to the `bound` keyword argument

and it must return a `h2mm_model` object.



<font color="brown">*As with passing a function to `print_func` it is not required, but recomended to keep the variable names*</font>

In [ ]:
def sample_bounds(new_model,current_model,old_model,bound):
    # like the print_func function, it's usually best to just keep the function signature the same
    # grab the obs matrix
    obs = new_model.obs
    # set first row of obs matrix to bound
    obs[0,:] = bound
    # change the obs matrix of the new model
    new_model.obs = obs
    # return the adjusted model
    return new_model

In [ ]:
bnd = np.array([0.09,0.01,0.9])
smp_bnd_opt_model_3state3 = EM_H2MM_C(init_model_3state3,color3,times3,bounds_func=sample_bounds,bounds=bnd)

<a id='7'></a>
# 7. Apendices


<a id='71'></a>
## 7.1 `h2mm_model` fields

Here is a list, and description of all the properties you can access.

so you can look at the model through its different attributes:
_(the following will produce `numpy` arrays as outputs)_
- `.prior` the optimized initial probability
- `.trans` the optimized transition probability matrix
- `.obs` the optimized emission probability matrix

You can see how many **states**, **streams** and *free parameters* are in teh model with the following
- `.nstate` the number of states in the model, this is the 0 dimension of `.prior`, `.trans` and `.obs` arrays
- `.ndet` the number of photon streams in the model, the 1 dimension of the `.obs` array
- <font color="green">`.k` the number of free parameters- this is $q^2+q*(r-1)-1$ where $q$ equals `.nstate` and $r$ equals `.ndet`.</font>

There are a number of other properties accesible, only after a model has been optimized:
- `.loglik` the calculated logliklihood of the model, a `float`
- `.bic` the Bayesian Information Criterion, a `float`
- <font color="green">`.nphot` the number of photons in the data set on which the model was optimized, an `int`</font>
- <font color="green">`.niter` the number of iterations used to optimize the model, ie either the maximum number of iterations or the iteration at which the loglik improvement was smaller than the minimum difference specified in `converged_min`</font>
A few other properties also exist, and tell you about how the model was optimized or *loglikelihood* calculated:

- The following return a boolean `True` or `False`   
    - <font color="green">`.is_conv` returns True or False depending on whether or not the model reached maximum loglikelihood, or if the maximum number iterations/time was reached, false for initial models, ones in the middle of optimization, or when their *loglikelihood* has been calculated without optimization</font>
    - <font color="green">`.is_opt` returns True or false depending on whether the model has been optimized- False if the model is an "initial" model, or the *loglikelihood* calculation was done without optimization</font>
    - <font color="green">`.is_calc`returns True if the loglik has been calculated against a set of data, and False if the model is an "initial model"</font>
- The remaining return an int or a string respectively 
    - <font color="green">`.conv_code` returns an int, representing the status of the model, ie has it been optimized with **data** did it converge etc.</font>
    - <font color="green">`.conv_str` returns a string the informs the user of if/how the particular object of the `h2mm_model` class finished it's optimization, this is basically a human readable interpretation of `.conv_code`</font>

The fields higlighed in <font color="green">green</font> above were not introduced before in this tutorial. This is because they are either self explanatory, or because a use for them was not immediatly obvious. The `H2MM_C` API was designed to be very full featured, so potentially useful fields were included in addition to fields  

### Rundown of fields we have not seen yet

<a id='711'></a>
### 7.1.1 Statistical Fields

Recall that the $BIC$ is defined as: $BIC = -2 LL + K \ln(n)$. The $K$ and $n$ values are accesible direcltly via there one fields:
- `.k` = $K$
- `.nphot` = $n$

At a core level, the `.bic` field is generated by calculating it from the `.loglik`, `.k` and `.nphot` fields. `.nphot` could also be useful as a way to test if two models were optimized against the same set of data. It is unlikely that two datasets will have exactly the same number of photons. **NOTE: you can be assured two models were optimized against different data if `.nphot` is different, however, the converse is not true. However, if `.nphot` is the same, it is highly likely (not guaranteed) that they were optimized against the same set of data.**


#### Let's test this, if this is true, then we should be able to calculate the $BIC$ with `.loglik'`, `.k` and `.nphot` 

In [ ]:
-2* fopt_model_3state.loglik + fopt_model_3state.k * np.log(fopt_model_3state.nphot) == fopt_model_3state.bic

In [ ]:
-2* fopt_model_3state.loglik + fopt_model_3state.k * np.log(fopt_model_3state.nphot)

<a id='712'></a>
### 7.1.2 `.niter`

This field has no impact on any other value. It is just a record of how many iterations `EM_H2MM_C` took. Most optimizations converge to values that are within floating point error regardless of the initial model (as long as the initial models have the same number of **states**), but the number of iterations that they take will vary.

#### Try different `h2mm_model` objects that have been created in this notebook, and check their `.niter` values

In [ ]:
fopt_model_2state.niter, opt_model_4state.niter

<a id='713'></a>
### 7.1.3 Convergence Info

The remaining fields (replicated below) are all about identifying how a model was calculated, and if they have converged.
We'll use these in [section 7.2.1](#721) to decide about optimizing more.
- The following return a boolean `True` or `False`   
    - `.is_conv` returns True or False depending on whether or not the model reached maximum loglikelihood, or if the maximum number iterations/time was reached, false for initial models, ones in the middle of optimization, or when their *loglikelihood* has been calculated without optimization
    - `.is_opt` returns True or false depending on whether the model has been optimized- False if the model is an "initial" model, or the *loglikelihood* calculation was done without optimization
    - `.is_calc`returns True if the loglik has been calculated against a set of data, and False if the model is an "initial model"
- The remaining return an int or a string respectively 
    - `.conv_code` returns an int, representing the status of the model, ie has it been optimized with **data** did it converge etc.
        - 0: initial models
        - 1: models in the middle of optimization whose *loglikelihood* has been calculated- this is generally only seen for models passed to the `print_func` function inside `EM_H2MM_C`
        - -1: models  in the middle of optimization whose *loglikelihood* has NOT been calculated- this is generally only seen in the `new_model` argument passed to the `print_func` function inside `EM_H2MM_C`
        - 2: unoptimized models whose *loglikelihood* has been calculated, this occurs for models whose *loglikelihood* has been calculated using `H2MM_arr` or the `.evaluate` method.
        - 3: Models that converged based on `converged_min` bounds
        - 4: Models that were optimized the `max_iter` number of iterations
        - 5: models that were optmized for the `max_time` amount of time
        - 6: models where an error occured in the subsequent iteration
    - `.conv_str` returns a string the informs the user of if/how the particular object of the `h2mm_model` class finished it's optimization, this is basically a human readable interpretation of `.conv_code`

**NOTE**: The `.conv_code` of 6 is rare, and these models should be used carefully, as they are the last good model before an error occured in the optimization, usually due to a floating point value being too close to zero, resulting in a divide by 0 error.

<a id='72'></a>
## 7.2 `EM_H2MM_C` and `.optimize` Arguments and Fields

`EM_H2MM_C` has some optional keyword parameters, which fall into three categories:
1. Processor settings: 
    - `num_cores=os.cpu_count()//2` sets the number of processors to use in the calculation, changing this will not affect the result of the calculation, only the speed of computation. The defaul (`os/cpu_count()//2`) finds the number of cpu cores that python detects, and havles it, as python usually detects virtual cores, and thus it is assumed that the actual number of physical cores is half that. This however is only true for hyperthreaded systems, and thus if your system is otherwise, you may want to change this setting.
2. Convergence criterions:
    - `max_iter=3600` the maximum number of iterations the optimization will continue unless another convergence criterion is met
    - `max_time=np.inf` the maximum amount of time the optimization will proceed, this is set to inf, so there will never be a timeout. It should be noted that this time is assesed by the C-level clock, which is often inaccurate, so while the time is supposedly in seconds, if this setting is set to a finite value, the optimization will likely terminate sooner than expected
    - `converged_min=1e-14` the minimum difference between models for an optimization to be considered converged, ie if the improvement in logliklihood is less than this, the optimization will terminate. This is the most common termination criteriod, as no futher optimization will proceed
    - <font color="green">`reset_niter=True` sets whether or not to set the number of iterations to 0 at the beginning of optimization. If set to `False`, then `max_iter` will trigger not after `max_iter` iterations have been perfomred in the current optimization, but rather it will start the counter at whatever number the input model has as `.niter`. Note also that if a data set with a different number of photons is passed, then regardless of whether `reset_niter` is set to `True` or `False`, the iteration counter will be reset.</font>
3. Printing options
    - `print_func='console'` 
4. Bounding inputs: methods of limiting the values and adjusting the functionality of the H<sup>2</sup>MM optimization
    - `bounds_func=None` the function to run be run after each iteration of the optimzation, used to bound or otherwise change the next model in the iteration. Built-in functions identified with strings (disucssed later), if given a Python function, that function will be run (also discussed later)
    - `bounds=None` additional input to the bounds_func function (discussed later)

<a id='721'></a>
### 7.2.1 The `reset_niter` argument

It is possible to feed an already optimized model back into `EM_H2MM_C`. This might be done for instance if `EM_H2MM_C` reached the maximum number of iterations, but the model doesn't seem right, and so we want to let it optimize some more.

By default, when `reset_niter` is `True`, `EM_H2MM_C` counts the number of iterations from 0, however, by setting `reset_niter` to `False` it will count iterations from the `.niter` value in the initial model passed to `EM_H2MM_C`. The same logic applies to the `.optimize` method of the `h2mm_model` object.

`reset_niter=False` should be used when a model returned after reaching the maximum number of iterations is to be optimized further, and you want the output model to reflect the cumulative number of iterations. When this is done however, `max_iter` must be increased each time the model is resubmitted for further optimization, otherwise the optimization will stop after the first round.

#### Keeping `.niter` after multiple rounds of optimization

In [ ]:
opt_multi_model_3state3 = EM_H2MM_C(init_model_3state3,color3,times3,max_iter=100,reset_niter=False)
opt_multi_model_3state3.niter

In [ ]:
opt_multi_model_3state3 = EM_H2MM_C(opt_multi_model_3state3,color3,times3,max_iter=200,reset_niter=False)
opt_multi_model_3state3.niter

Note how the `.niter` field is smaller if we do the same, but with `reset_niter=True`

In [ ]:
opt_multi_model_3state3 = EM_H2MM_C(init_model_3state3,color3,times3,max_iter=110,reset_niter=True)
opt_multi_model_3state3 = EM_H2MM_C(opt_multi_model_3state3,color3,times3,max_iter=100,reset_niter=True)
opt_multi_model_3state3.niter